## Extracted pages stats

In [1]:
#!/usr/bin/env python3

import pandas as pd
import os

# Define the path to the CSV file
CSV_FILE_PATH = "/home/omar/projects/haystack/tablequest/metadata/sampled_pages_metadata.csv"

def analyze_csv(file_path):
    """
    Analyzes the provided CSV file and prints various statistics.
    """
    if not os.path.exists(file_path):
        print(f"Error: CSV file not found at {file_path}")
        return

    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return

    print(f"Analysis of: {file_path}\n")

    # --- Overall Statistics ---
    print("--- Overall Statistics ---")
    total_pages = len(df)
    print(f"Total pages processed (rows in CSV): {total_pages}")

    unique_documents = df['document_name'].nunique()
    print(f"Total unique documents: {unique_documents}")

    unique_companies = df['company_name'].nunique()
    print(f"Total unique companies: {unique_companies}")
    print("-" * 30)

    # --- Document Statistics ---
    print("--- Document Statistics ---")
    if unique_documents > 0:
        avg_pages_per_document = total_pages / unique_documents
        print(f"Average pages extracted per document: {avg_pages_per_document:.2f}")
    
    tables_per_document = df.groupby('document_name')['table_count'].sum()
    print(f"Total tables across all documents: {tables_per_document.sum()}")
    if not tables_per_document.empty:
        print(f"Average tables per document: {tables_per_document.mean():.2f}")
        print(f"Min tables in a document: {tables_per_document.min()}")
        print(f"Max tables in a document: {tables_per_document.max()}")
    print("-" * 30)

    # --- Company Statistics ---
    print("--- Company Statistics ---")
    company_doc_counts = df.groupby('company_name')['document_name'].nunique()
    print("Number of unique documents per company:")
    print(company_doc_counts.sort_values(ascending=False))
    print()

    company_page_counts = df['company_name'].value_counts()
    print("Number of pages processed per company:")
    print(company_page_counts)
    print()

    company_table_counts = df.groupby('company_name')['table_count'].sum()
    print("Total tables per company:")
    print(company_table_counts.sort_values(ascending=False))
    print()
    
    if not company_page_counts.empty and not company_table_counts.empty:
        # Ensure alignment for division, reindex company_page_counts if necessary
        avg_tables_per_page_company = company_table_counts / company_page_counts.reindex(company_table_counts.index)
        print("Average tables per page for each company:")
        print(avg_tables_per_page_company.sort_values(ascending=False).dropna())
    print("-" * 30)

    # --- Report Type Statistics ---
    print("--- Report Type Statistics ---")
    report_type_counts = df['report_type'].value_counts()
    print("Distribution of report types:")
    print(report_type_counts)
    print("-" * 30)

    # --- Report Year Statistics ---
    print("--- Report Year Statistics ---")
    df['report_year_str'] = df['report_year'].astype(str) # Handle mixed types like '2021' and '2021Q1'
    report_year_counts = df['report_year_str'].value_counts().sort_index()
    print("Distribution of report years/periods:")
    print(report_year_counts)
    print("-" * 30)

    # --- Page Number Statistics ---
    print("--- Page Number Statistics ---")
    print(f"Min page number extracted: {df['page_number'].min()}")
    print(f"Max page number extracted: {df['page_number'].max()}")
    print(f"Mean page number extracted: {df['page_number'].mean():.2f}")
    print(f"Median page number extracted: {df['page_number'].median():.2f}")
    print("-" * 30)
    
    # --- Table Count Statistics (per page) ---
    print("--- Table Count (per page) Statistics ---")
    print(f"Total tables found across all pages: {df['table_count'].sum()}")
    print(f"Min tables on a page: {df['table_count'].min()}")
    print(f"Max tables on a page: {df['table_count'].max()}")
    print(f"Mean tables per page: {df['table_count'].mean():.2f}")
    print(f"Median tables per page: {df['table_count'].median():.2f}")
    table_count_distribution = df['table_count'].value_counts().sort_index()
    print("\nDistribution of table counts per page:")
    print(table_count_distribution)
    print("-" * 30)

if __name__ == "__main__":
    analyze_csv(CSV_FILE_PATH)

Analysis of: /home/omar/projects/haystack/tablequest/metadata/sampled_pages_metadata.csv

--- Overall Statistics ---
Total pages processed (rows in CSV): 1580
Total unique documents: 22
Total unique companies: 22
------------------------------
--- Document Statistics ---
Average pages extracted per document: 71.82
Total tables across all documents: 2376
Average tables per document: 108.00
Min tables in a document: 73
Max tables in a document: 220
------------------------------
--- Company Statistics ---
Number of unique documents per company:
company_name
3M                    1
ACTIVISIONBLIZZARD    1
VERIZON               1
PFIZER                1
PEPSICO               1
MGMRESORTS            1
KRAFTHEINZ            1
JPMORGAN              1
JOHNSON_JOHNSON       1
GENERALMILLS          1
CVSHEALTH             1
CORNING               1
COCACOLA              1
BOEING                1
BLOCK                 1
BESTBUY               1
AMERICANWATERWORKS    1
AMERICANEXPRESS       1
AMCOR 

## Sampled pages stats

In [3]:
import os
import pandas as pd

# Path where sampled pages are stored, one subfolder per PDF (named without .pdf)
SAMPLED_PAGES_DIR = "/home/omar/projects/haystack/tablequest/sampled_pages"

records = []
for base_name in os.listdir(SAMPLED_PAGES_DIR):
    subdir = os.path.join(SAMPLED_PAGES_DIR, base_name)
    if not os.path.isdir(subdir):
        continue
    # count all files (e.g. PNGs) in that subfolder
    count = sum(
        1 for fname in os.listdir(subdir)
        if os.path.isfile(os.path.join(subdir, fname))
    )
    # reconstruct original file name (add .pdf) or leave as-is
    pdf_name = f"{base_name}.pdf"
    records.append({"file_name": pdf_name, "tables_count": count})

# build and display DataFrame
df = pd.DataFrame(records)
df = df.sort_values("file_name").reset_index(drop=True)
df.sort_values("tables_count", ascending=False, inplace=True)
print(df['tables_count'].sum())
df


559


,file_name,tables_count
15,JPMORGAN_2021Q1_10Q.pdf,60
13,GENERALMILLS_2019_10K.pdf,36
16,KRAFTHEINZ_2019_10K.pdf,34
3,AES_2022_10K.pdf,33
19,PFIZER_2021_10K.pdf,30
8,BLOCK_2020_10K.pdf,28
10,COCACOLA_2017_10K.pdf,27
20,VERIZON_2021_10K.pdf,26
9,BOEING_2018_10K.pdf,26
21,WALMART_2019_10K.pdf,24


## enriched metadat

In [1]:
import pandas as pd

meta = pd.read_csv('/home/omar/projects/haystack/tablequest/metadata/sampled_pages_meta_enriched.csv')
meta.head()

,image_name,company_name,report_type,report_year,page_number,table_count,tables_markdown,min_cells
0,CORNING_2022_10K_page_32.png,CORNING,10K,2022,32,2,| ...,91.0
1,CORNING_2022_10K_page_35.png,CORNING,10K,2022,35,2,| | ...,9.0
2,CORNING_2022_10K_page_42.png,CORNING,10K,2022,42,3,| ...,6.0
3,CORNING_2022_10K_page_75.png,CORNING,10K,2022,75,4,| | December 31...,6.0
4,CORNING_2022_10K_page_76.png,CORNING,10K,2022,76,3,| | December 3...,12.0


In [5]:
meta[meta['image_name'] == 'CORNING_2022_10K_page_32.png']


,image_name,company_name,report_type,report_year,page_number,table_count,tables_markdown,min_cells
0,CORNING_2022_10K_page_32.png,CORNING,10K,2022,32,2,| ...,91.0


## Generated answers